<a href="https://colab.research.google.com/github/feknall/machinelearning-playground/blob/main/Gras_Winter2022_Final_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download and Setup

In [ ]:
!pip install kaggle

In [ ]:
!mkdir /root/.kaggle
!touch /root/.kaggle/kaggle.json

In [ ]:
api_token = {"username":"hfkh77","key":"1798ff5c420650f0570e35bf05ef0a3b"}

import json
import zipfile
import os
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!cat /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions list

In [ ]:
!kaggle competitions download -c severstal-steel-defect-detection

In [ ]:
!unzip /content/drive/MyDrive/ai-gras-2/severstal-steel-defect-detection.zip -d /content/drive/MyDrive/ai-gras-2/dataset

In [ ]:
!ls -l /content/drive/MyDrive/ai-gras-2/dataset/train_images | wc -l

In [ ]:
!ls -l /content/drive/MyDrive/ai-gras-2/dataset/test_images | wc -l

In [ ]:
!pip install --upgrade pandas_profiling

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
%matplotlib inline

#Insight

In [ ]:
dataset_path = '/content/drive/MyDrive/ai-gras-2/dataset'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import skimage.io
import os 
import tqdm
import glob
import tensorflow 
import warnings
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import keras
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import grey2rgb
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from tensorflow.keras.optimizers import Adam
from keras.applications.nasnet import NASNetLarge, NASNetMobile
from keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K
from tensorflow.keras.applications.xception import Xception, preprocess_input
#import tensorflow_addons as tfa
#from tensorflow.keras.metrics import Metric
#from tensorflow_addons.utils.types import AcceptableDTypes, FloatTensorLike
from typeguard import typechecked
from typing import Optional
import pickle 

In [ ]:
train_size, test_size = 12568, 5506

def func(v, p):
    a = p * sum(v) / 100
    return "{:.2f}%\n({:.0f})".format(p, a)

labels = ['train', 'test']
sizes = [train_size, test_size]
explode = (0.1, 0.1)
fig, ax = plt.subplots(figsize=(32, 16))
ax.pie(sizes, explode=explode, labels=labels, textprops={'fontsize': 28}, autopct=lambda p: func(sizes, p), shadow=True)
# fig.suptitle('Defect Distribution', fontsize=25, fontweight='bold')
# fig.set_facecolor("tan")
plt.show()


In [ ]:
df = pd.read_csv(f"{dataset_path}/train.csv")

In [ ]:
df

In [ ]:
df.shape

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df)

In [ ]:
class_id_count = df.groupby(['ClassId']).count()

In [ ]:
class_id_count

In [ ]:
class_id_count.columns

In [ ]:
class_id_count.shape

In [ ]:
import matplotlib.pyplot as plt

labels = range(1, class_id_count.shape[0] + 1)
sizes = class_id_count['ImageId']

fig1, ax1 = plt.subplots(figsize=(10,10))
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', 
        startangle=90, textprops={'fontsize': 16})

plt.show()

In [ ]:
import matplotlib.pyplot as plt

labels = [str(i) for i in range(1, class_id_count.shape[0] + 1)]
sizes = class_id_count['ImageId']

fig1, ax1 = plt.subplots(figsize=(10,10))
ax1.bar(labels, sizes)
ax1.set_xlabel("Class")
ax1.set_ylabel("Count")

print("Size: " + sizes)
plt.show()

In [ ]:
tqdm(df.values[:10])

In [ ]:
limit = 1000000

image_list = []
class_list = []
name_list = []
for img, ClassId, EncodedPixels in tqdm(df.values[:limit]):
    image = cv2.imread(f"{dataset_path}/train_images/{img}", cv2.IMREAD_COLOR)
    image = cv2.resize(image, (80, 80))
    # image = cv2.resize(image, fx=0.5, fy=0.5)
    image_list.append(image)
    class_list.append(ClassId)
    name_list.append(img)

In [ ]:
class_list_file_name = f"{dataset_path}/class_list.pkl"

In [ ]:
# open_file = open(class_list_file_name, "wb")
# pickle.dump(class_list, open_file)
# open_file.close()

In [ ]:
open_file = open(class_list_file_name, "rb")
class_list = pickle.load(open_file)
open_file.close()

print(len(class_list))

In [ ]:
name_list_file_name = f"{dataset_path}/name_list.pkl"

In [ ]:
# open_file = open(name_list_file_name, "wb")
# pickle.dump(name_list, open_file)
# open_file.close()

In [ ]:
open_file = open(name_list_file_name, "rb")
name_list = pickle.load(open_file)
open_file.close()

print(len(name_list))

In [ ]:
image_list_file_name = f"{dataset_path}/image_list_80_80.pkl"

In [ ]:
# open_file = open(image_list_file_name, "wb")
# pickle.dump(image_list, open_file)
# open_file.close()

In [ ]:
open_file = open(image_list_file_name, "rb")
image_list = pickle.load(open_file)
open_file.close()

print(len(image_list))

In [ ]:
num_images = (5, 5)
total_images = num_images[0] + num_images[1]
fig, ax = plt.subplots(num_images[0], num_images[1], figsize=(10*total_images, 10*total_images))
for i in range(num_images[0]):
  for j in range(num_images[1]):
    id = i * num_images[1] + j
    print(f"id: {id}, class: {class_list[id]}, name: {name_list[id]}")
    ax[i, j].imshow(image_list[id])

In [ ]:
class_list

In [ ]:
X = np.array(image_list)
X = X / 255

In [ ]:
from sklearn.preprocessing import OneHotEncoder

y = np.array(class_list).reshape(1, -1)
encoder = LabelEncoder()
y = encoder.fit_transform(class_list)
y = to_categorical(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
import time 

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
import sklearn

METRICS = [
           tf.keras.metrics.BinaryAccuracy(name="accuracy"),
           tf.keras.metrics.Precision(name="precision"),
           tf.keras.metrics.Recall(name="recall"),
           tf.keras.metrics.AUC(name="auc"),
           f1_score
]

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss', patience = 20, verbose = 1, factor = 0.50, min_lr = 1e-10)
mcp = ModelCheckpoint('model.h5')
es = EarlyStopping(verbose=1, patience=20)
time_callback = TimeHistory()

## Xception

In [ ]:
xception = tf.keras.applications.Xception(input_shape=(80, 80, 3), include_top=False, weights="imagenet")

In [ ]:
for layer in xception.layers[:-5]:
  layer.trainable=False

In [ ]:
xception_model = Sequential()
xception_model.add(xception)
xception_model.add(Dropout(0.5))
xception_model.add(Flatten())
xception_model.add(Dense(256, activation="relu"))
xception_model.add(Dropout(0.3))
xception_model.add(Dense(128, activation="relu"))
xception_model.add(Dropout(0.3))
xception_model.add(Dense(256, activation="relu"))
xception_model.add(Dense(4, activation="softmax"))

In [ ]:
xception_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=METRICS)

In [ ]:
%time
xception_history = xception_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64, verbose=1, callbacks=[lrd, mcp, es, time_callback])

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(xception, to_file="xception.png", show_shapes=True, show_layer_names=True)
Image(filename="xception.png")

In [ ]:
times = time_callback.times
print(times)

In [ ]:
with open(f'{dataset_path}/xception_epoch_20_80_80.pkl', 'wb') as file_pi:
        pickle.dump(xception_history.history, file_pi)

In [ ]:
xception_model.save(f'{dataset_path}/model_xception_epoch_20_80_80')

## ResNet50

In [ ]:
resnet = tf.keras.applications.ResNet50(input_shape=(80, 80, 3), include_top=False, weights="imagenet")

In [ ]:
for layer in resnet.layers[:-5]:
  layer.trainable=False

In [ ]:
resnet_model = Sequential()
resnet_model.add(resnet)
resnet_model.add(Dropout(0.5))
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation="relu"))
resnet_model.add(Dropout(0.3))
resnet_model.add(Dense(128, activation="relu"))
resnet_model.add(Dropout(0.3))
resnet_model.add(Dense(256, activation="relu"))
resnet_model.add(Dense(4, activation="softmax"))

In [ ]:
resnet_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=METRICS)

In [ ]:
%time
resnet_history = resnet_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64, verbose=1, callbacks=[lrd, mcp, es, time_callback])

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(resnet, to_file="resnet.png", show_shapes=True, show_layer_names=True)
Image(filename="resnet.png")

In [ ]:
times = time_callback.times
print(times)

In [ ]:
with open(f'{dataset_path}/resnet_epoch_20_80_80.pkl', 'wb') as file_pi:
        pickle.dump(resnet_history.history, file_pi)

In [ ]:
resnet_model.save(f'{dataset_path}/model_resnet_epoch_20_80_80')

In [ ]:
resnet_model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(resnet_model, to_file="resnet_model.png", show_shapes=True, show_layer_names=True)
Image(filename="resnet_model.png")

## Simple

In [ ]:
model_simple = Sequential()
model_simple.add(Conv2D(16, (3, 3), activation='relu'))
model_simple.add(MaxPooling2D((2, 2)))
model_simple.add(Conv2D(32, (3, 3), activation='relu'))
model_simple.add(MaxPooling2D((2, 2)))
model_simple.add(Flatten())
model_simple.add(Dense(16, activation='relu'))
model_simple.add(Dense(16, activation='relu'))
model_simple.add(Dropout(0.3))
model_simple.add(Dense(4, activation="softmax"))

In [ ]:
model_simple.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=METRICS)

In [ ]:
%time
history_simple = model_simple.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64, verbose=1, callbacks=[lrd, mcp, es, time_callback])

In [ ]:
model_simple.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(model_simple, to_file="convnet.png", show_shapes=True, show_layer_names=True)
Image(filename="convnet.png")

In [ ]:
times = time_callback.times
print(times)

In [ ]:
with open(f'{dataset_path}/simple_epoch_20_80_80.pkl', 'wb') as file_pi:
        pickle.dump(history_simple.history, file_pi)

In [ ]:
model_simple.save(f'{dataset_path}/model_simple_epoch_20_80_80')

## Common

In [ ]:
times_xception = [251.20164227485657, 230.650208234787, 230.2002878189087, 229.25759291648865, 230.1872398853302, 228.3032259941101, 228.92920637130737, 228.53148126602173, 226.7629246711731, 226.5128927230835, 227.91919088363647, 227.28745412826538, 227.61123967170715, 227.70413732528687, 227.7826268672943, 227.32461977005005, 228.01096487045288, 226.33702516555786, 226.996075630188, 226.78124475479126]
times_resnet = [170.07486009597778, 158.72148513793945, 158.89877462387085, 158.84877133369446, 158.47053003311157, 158.5059313774109, 158.54066944122314, 158.60045862197876, 158.33677172660828, 158.79621529579163, 158.31911182403564, 158.33913111686707, 158.55075979232788, 158.62329816818237, 158.23744988441467, 158.24436902999878, 158.09473824501038, 158.22466158866882, 158.36276364326477, 158.6155173778534]
times_normal = [30.092936038970947, 29.78129005432129, 25.5189368724823, 24.822543144226074, 25.094614028930664, 24.917269229888916, 25.530674934387207, 25.31166958808899, 25.527685165405273, 24.840967416763306, 24.882030487060547, 25.337437391281128, 24.880992650985718, 24.911091804504395, 24.83761978149414, 24.741199254989624, 24.787998914718628, 24.885679244995117, 24.84987711906433, 24.846619129180908]

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

ax.bar(range(1, len(times_xception) + 1), times_xception)
ax.bar(range(1, len(times_resnet) + 1), times_resnet)
ax.bar(range(1, len(times_normal) + 1), times_normal)

ax.set_xlabel('Epochs')
ax.set_ylabel('Time')
ax.legend(['Xception', 'ResNet', 'Normal'])

plt.show()

In [ ]:
xception_history_loaded = pickle.load(open(f'{dataset_path}/xception_epoch_20_80_80.pkl', "rb"))

In [ ]:
resnet_history_loaded = pickle.load(open(f'{dataset_path}/resnet_epoch_20_80_80.pkl', "rb"))

In [ ]:
normal_history_loaded = pickle.load(open(f'{dataset_path}/simple_epoch_20_80_80.pkl', "rb"))

In [ ]:
legends = ['Xception - training', 'Xception - validation', 'ResNet - training', 'ResNet - validation', 'Normal - training', 'Normal - validation']

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

acc, val_acc = xception_history_loaded['accuracy'], xception_history_loaded['val_accuracy']
ax.plot(range(1, len(acc) + 1), acc)
ax.plot(range(1, len(val_acc) + 1), val_acc)

acc, val_acc = resnet_history_loaded['accuracy'], resnet_history_loaded['val_accuracy']
ax.plot(range(1, len(acc) + 1), acc)
ax.plot(range(1, len(val_acc) + 1), val_acc)

acc, val_acc = normal_history_loaded['accuracy'], normal_history_loaded['val_accuracy']
ax.plot(range(1, len(acc) + 1), acc)
ax.plot(range(1, len(val_acc) + 1), val_acc)

ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.legend(legends)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

loss, val_loss = xception_history_loaded['loss'], xception_history_loaded['val_loss']
ax.plot(range(1, len(loss) + 1), loss)
ax.plot(range(1, len(val_loss) + 1), val_loss)

acc, val_acc = resnet_history_loaded['loss'], resnet_history_loaded['loss']
ax.plot(range(1, len(acc) + 1), acc)
ax.plot(range(1, len(val_acc) + 1), val_acc)

loss, val_loss = history_simple.history['loss'], history_simple.history['val_loss']
ax.plot(range(1, len(loss) + 1), loss)
ax.plot(range(1, len(val_loss) + 1), val_loss)

ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend(legends)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

loss, val_loss = xception_history_loaded['auc'], xception_history_loaded['val_auc']
ax.plot(range(1, len(loss) + 1), loss)
ax.plot(range(1, len(val_loss) + 1), val_loss)

acc, val_acc = resnet_history_loaded['auc'], resnet_history_loaded['val_auc']
ax.plot(range(1, len(acc) + 1), acc)
ax.plot(range(1, len(val_acc) + 1), val_acc)

auc, val_auc = history_simple.history['auc'], history_simple.history['val_auc']
ax.plot(range(1, len(auc) + 1), auc)
ax.plot(range(1, len(val_auc) + 1), val_auc)

ax.set_xlabel('Epochs')
ax.set_ylabel('AUC')
ax.legend(legends)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

loss, val_loss = xception_history_loaded['precision'], xception_history_loaded['val_precision']
ax.plot(range(1, len(loss) + 1), loss)
ax.plot(range(1, len(val_loss) + 1), val_loss)

acc, val_acc = resnet_history_loaded['precision'], resnet_history_loaded['val_precision']
ax.plot(range(1, len(acc) + 1), acc)
ax.plot(range(1, len(val_acc) + 1), val_acc)

precision, val_precision = history_simple.history['precision'], history_simple.history['val_precision']
ax.plot(range(1, len(precision) + 1), precision)
ax.plot(range(1, len(val_precision) + 1), val_precision)

ax.set_xlabel('Epochs')
ax.set_ylabel('Precision')
ax.legend(legends)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))

loss, val_loss = xception_history_loaded['f1_score'], xception_history_loaded['val_f1_score']
ax.plot(range(1, len(loss) + 1), loss)
ax.plot(range(1, len(val_loss) + 1), val_loss)

acc, val_acc = resnet_history_loaded['f1_score'], resnet_history_loaded['val_f1_score']
ax.plot(range(1, len(acc) + 1), acc)
ax.plot(range(1, len(val_acc) + 1), val_acc)

f1, val_f1 = history_simple.history['f1_score'], history_simple.history['val_f1_score']
ax.plot(range(1, len(f1) + 1), f1)
ax.plot(range(1, len(val_f1) + 1), val_f1)

ax.set_xlabel('Epochs')
ax.set_ylabel('F1-Score')
ax.legend(legends)

plt.show()

## Test

In [ ]:
evaluate_resnet = resnet_model.evaluate(X_test, y_test)

In [ ]:
evaluate_resnet

In [ ]:
evaluate_xception = xception_model.evaluate(X_test, y_test)

In [ ]:
evaluate_xception

In [ ]:
evaluate_simple = model_simple.evaluate(X_test, y_test)

In [ ]:
evaluate_simple

In [ ]:
del len

In [ ]:
m = ['Loss', 'Accuracy', 'Precision', 'Recall', 'AUC', 'F1 Score']
fig, ax = plt.subplots(figsize=(16, 12))

X_axis = np.arange(len(m))

ax.bar(X_axis - 0.2, evaluate_xception, 0.2)
ax.bar(X_axis, evaluate_resnet, 0.2)
ax.bar(X_axis + 0.2, evaluate_simple, 0.2)

plt.xticks(X_axis, m)

ax.set_xlabel('Metric')
ax.set_ylabel('Value')
ax.legend(['Xception', 'ResNet', 'Normal'])

plt.show()

In [ ]:
import numpy as np

df_submission = pd.read_csv(f'{dataset_path}/sample_submission.csv')
print(df_submission.shape)
df_submission.head()

In [ ]:
df_submission.to_csv("submission.csv", index=False)
df_submission